<a href="https://colab.research.google.com/github/TurgutOzkan/DataSciencity/blob/master/How_to_Obtain_Coefficients_in_Linear_Regression%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this post, we will go through the technical details of deriving parameters for linear regression. The post will directly dive into linear algebra and matrix representation of a linear model and show how to obtain coefficients without using the of-the-shelf Scikit-learn linear estimator.

Let's formulate our linear regression in the following :

$  Y_i = \beta_1 +  \beta_2 X_{2i} + \beta_3  X_{3i} + ...\beta_k X_{ki} +  e_i \tag{1} $

where $\beta$s are coefficients and $e$ is the error term.

Using matrix notation, the above can be simplified as:

$y = X \beta + e  \tag{2} $

In order to better visualize and understand from linear algebraic perspective, let's express our equation in matrix form.

$\begin{bmatrix} Y_1\\Y_2 \\.\\. \\Y_n\end{bmatrix} =
\begin{bmatrix}1 & X_{21} & X_{31}& .. & X_{k1}\\
1 & X_{22} & X_{32} & .. &X_{k2}\\
. & . & . & . & .\\
1 & X_{2n} & X_{3n}& .. & X_{kn}
\end{bmatrix} 
\begin{bmatrix} \beta_1 \\ \beta_2 \\ .. \\ \beta_k
\end{bmatrix} +
\begin{bmatrix} \epsilon_1\\ \epsilon_2 \\. \\ \epsilon_n\end{bmatrix}  \tag{3}$  

Using ordinary least squares (OLS), we will obtain $\beta$s that will minimize the squared error. Below is simply $y - \hat{y}$ squared, in other words, predicted values are subtracted from the observed values and then squared:

$
\sum{\epsilon_i}^2 = \sum(Y_i - \beta_1  - \beta_2X_{2i}  \space - \space... \space - \beta_kX_{ki})^2 \tag{4}
$

Expressing the squared errors ($\sum{\epsilon_i}^2)$ in matrix notation, we can express the same thing as $\epsilon^T \epsilon$ --transposing the error vector since:

 $\epsilon^T \epsilon = [\epsilon_1 \space \epsilon_2 \space.. \space \epsilon_n] \begin{bmatrix} \epsilon_1\\\epsilon_2 \\.\\. \\\epsilon_n\end{bmatrix} =  \epsilon_1^2 + \epsilon_2^2 +.. \space \epsilon_n^2  = \sum\epsilon_i^2 \tag{5}$

From Equation (2):

$ \epsilon = y - X\beta \tag{6}$
$\epsilon^T \epsilon = (y - X\beta)^T(y- X\beta) \tag{7}$
$=y^Ty - 2\beta^TX^Ty + \beta^TX^TX\beta \tag{8}$

Note that we used distribution property of transpose when distributing the transpose sign in the paranthesis -- $(X\beta)^T = \beta^TX^T $. Also, note that $\beta^TX^Ty$ is a scalar number so it is equal to its transpose $y^TX\beta.$ That is how we got the $2\beta^TX^Ty$ above.

Now is the critical part. We will take partial derivates of Equation (8) with respect to $\beta$. More formally:

$\frac{\Large \partial (\epsilon^T \epsilon)}{\Large \partial \beta} = - 2X^Ty + 2X^TX\beta \tag{9}$

and we set the above equation to zero because we want to obtain the global minimum where the slope is zero. Doing so:

$(X^TX)\beta = X^Ty \tag{10}$, which can be expressed as:

$\beta = (X^TX)^{-1} \space X^Ty \tag{11} $

Please note that Equation (11) yields the coefficients of our regression line if there is an inverse for $ (X^TX)$.




#Implementation

Now, let's test above equations within a code and compare it with scikit-learn results. For this test, we will use Scikitlearn Boston Housing data set.

In [ ]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
X, y = load_boston(return_X_y=True)
print(X.shape, y.shape)

(506, 13) (506,)


In [ ]:
# Define a functin for our own implementation.
def get_regression_coefs(X, y):
    """Takes a feature matrix and adds 1s as intercept, and then
    apply partial derivative on the squared errors
    """
    X = np.c_[ np.ones(X.shape[0]), X]
    XTX = ((X.T).dot(X))
    XTX_inv = np.linalg.inv(XTX)
    XTy = ((X.T).dot(y))
    print("intercept and slopes:", XTX_inv.dot(XTy) )

In [ ]:
get_regression_coefs(X, y)

intercept and slopes: [ 3.64594884e+01 -1.08011358e-01  4.64204584e-02  2.05586264e-02
  2.68673382e+00 -1.77666112e+01  3.80986521e+00  6.92224640e-04
 -1.47556685e+00  3.06049479e-01 -1.23345939e-02 -9.52747232e-01
  9.31168327e-03 -5.24758378e-01]


We obtained 36.45 as slope and other coefficients in order of they appear in our data. Now, let's see what scikitlearn will yield.

In [ ]:
reg = LinearRegression().fit(X, y)
reg.score(X, y)
print("slopes:", reg.coef_)
print("intercept:", reg.intercept_)

slopes: [-1.08011358e-01  4.64204584e-02  2.05586264e-02  2.68673382e+00
 -1.77666112e+01  3.80986521e+00  6.92224640e-04 -1.47556685e+00
  3.06049479e-01 -1.23345939e-02 -9.52747232e-01  9.31168327e-03
 -5.24758378e-01]
intercept: 36.459488385090125


Not surprisingly, we got the same result. In fact, Scikitlearn applies the same logic as Scipy the Ordinary Least Squares (scipy.linalg.lstsq). While one may not need to create every algorithm from scratch, for the aspiting data scientists, it is recommended to learn how to derive simple linear regression coefficients for one or more variables.

Source: Basic Econometrics by Gujarati & Porter